> Proszę zaimplementować algorytm QuickSort bez użycia rekurencji (ale można wykorzystać własny stos).

##### Funkcja testująca poprawność algorytmu

In [1]:
import random

def test_sort(sorting_fn, *, 
              samples=20,         # A number of tests that will be performed
              val_counts=(0, 50), # A minimum and maximum number of values to sort that will be generated
              range_=(-100, 100), # A range which will be used to create a random list of values from this range
              in_place=True,      # Information whether an algorithm runs in place or not
              failed_only=False,  # Show only failed tests. Works only if no_results is set to False
              print_out_fn=None,  # An user-defined function to print additional information. Works only if no_results is set to False
              no_results=False    # When set to True, no results will be printed (useful only for benchmark)
             ):                  
    passed = 0                   
    for i in range(samples):
        random_lst = [random.randint(*range_) for _ in range(random.randint(*val_counts))]
        random_lst_before = random_lst[:]
        expected = sorted(random_lst)
        output = sorting_fn(random_lst)
        if not in_place:
            random_lst = output
        is_correct = random_lst == expected
        passed += is_correct
        
        if not no_results:
            if not failed_only or (failed_only and not is_correct):
                print(f'TEST #{i+1}:')
                print(f'Before sorting: {random_lst_before}')
                print(f'After sorting: {random_lst}')
                print(f'Expected result: {expected}')
                print(f'Test {"PASSED" if is_correct else "FAILED"}')
                print(f'Current passed-to-tested ratio: {passed}/{i+1}')
                if print_out_fn:
                    print(f'========== Additional results after sorting  ==========')
                    print_out_fn(random_lst)
                print()
                
    if not no_results:
        print(f'Sorting algorithms is {"correct" if passed == samples else "wrong"}')
        print(f'Passed tests in total: {passed}/{samples}')

### Implementacja algorytmu #1 (ze stosem w postaci listy Pythonowej)

In [2]:
def quick_sort(arr):
    if len(arr) < 2: return
    stack = [(0, len(arr)-1)]
    
    while stack:
        # Take the last indices pair out of the stack
        left_idx, right_idx = stack.pop()
        
        pivot_position = _partition(arr, left_idx, right_idx)

        if pivot_position - left_idx < right_idx - pivot_position:
            # Store indices of the longer part at first if has at least 2 elements
            if pivot_position + 1 < right_idx:
                stack.append((pivot_position + 1, right_idx))
                # Then store indices of a shorter part (as it will produce less parts after sorting)
                # (if has at least 2 elements and we are sure the longer part has also at least 2 elements)
                if left_idx < pivot_position:
                    stack.append((left_idx, pivot_position))
        else:
            # Store indices of the longer part at first if has at least 2 elements
            if left_idx < pivot_position:
                stack.append((left_idx, pivot_position))
                # Then store indices of a shorter part (as it will produce less parts after sorting)
                # (if has at least 2 elements and we are sure the longer part has also at least 2 elements)
                if pivot_position + 1 < right_idx:
                    stack.append((pivot_position + 1, right_idx))
        
        
def _partition(arr, left_idx, right_idx):
    pivot = arr[left_idx]
    
    # Partition an array into 2 subarrays of elements lower than or
    # equal to a pivot and of elements greater than a pivot (in this
    # partition algorithm pivot isn't placed on a fixed position but
    # can be also swapped like all the remaining values)
    i = left_idx - 1
    j = right_idx + 1
    while True:
        i += 1
        while arr[i] < pivot: i += 1
            
        j -= 1
        while arr[j] > pivot: j -= 1
        
        if i < j:
            _swap(arr, i, j)
        else:
            return j  # Return a pivot position after the last swap

    
def _swap(arr, i, j):
    arr[i], arr[j] = arr[j], arr[i]

###### Kilka testów

In [3]:
test_sort(quick_sort, samples=100)

TEST #1:
Before sorting: [-75, 92, -74, -43, -97, -20, 40, 17, 12, -56, 20, -49, -28, -8, 3, -30, 71, 26, -2, -36, -89, -1, 22, 50]
After sorting: [-97, -89, -75, -74, -56, -49, -43, -36, -30, -28, -20, -8, -2, -1, 3, 12, 17, 20, 22, 26, 40, 50, 71, 92]
Expected result: [-97, -89, -75, -74, -56, -49, -43, -36, -30, -28, -20, -8, -2, -1, 3, 12, 17, 20, 22, 26, 40, 50, 71, 92]
Test PASSED
Current passed-to-tested ratio: 1/1

TEST #2:
Before sorting: [-89, 22, 99, 81, 65, 93, 89, -94, 87, 60, 58, -89, 39, 100, 51, -31, -71, -96, -39, 33, -12, 34, 50, 39, 0, 71, -67, 98, -66, -100, 15, -19, 80]
After sorting: [-100, -96, -94, -89, -89, -71, -67, -66, -39, -31, -19, -12, 0, 15, 22, 33, 34, 39, 39, 50, 51, 58, 60, 65, 71, 80, 81, 87, 89, 93, 98, 99, 100]
Expected result: [-100, -96, -94, -89, -89, -71, -67, -66, -39, -31, -19, -12, 0, 15, 22, 33, 34, 39, 39, 50, 51, 58, 60, 65, 71, 80, 81, 87, 89, 93, 98, 99, 100]
Test PASSED
Current passed-to-tested ratio: 2/2

TEST #3:
Before sorting: [-90

### Implementacja algorytmu #2 (ze stosem w postaci własnej listy odsyłaczowej zaimplementowanym obiektowo)

#### Implementacja stosu

In [4]:
class Node:
    def __init__(self, val=None):
        self.val = val
        self.next = None


class Stack:
    def __init__(self):
        self.top = None
        self.size = 0

    def __bool__(self):
        return self.size > 0
    
    def push(self, val: object):
        node = Node(val)
        if not self:
            self.top = node
        else:
            node.next = self.top
            self.top = node
        self.size += 1
        
    def pop(self) -> object:
        if not self:
            raise IndexError(f'pop from an empty {self.__class__.__name__}')
        removed = self.top.val
        self.top = self.top.next
        self.size -= 1
        return removed

#### Implementacja algorytmu sortowania

In [5]:
def quick_sort(arr):
    if len(arr) < 2: return
    stack = Stack()
    stack.push((0, len(arr)-1))
    
    while stack:
        # Take the last indices pair out of the stack
        left_idx, right_idx = stack.pop()
        
        pivot_position = _partition(arr, left_idx, right_idx)

        if pivot_position - left_idx < right_idx - pivot_position:
            # Store indices of the longer part at first if has at least 2 elements
            if pivot_position + 1 < right_idx:
                stack.push((pivot_position + 1, right_idx))
                # Then store indices of a shorter part (as it will produce less parts after sorting)
                # (if has at least 2 elements and we are sure the longer part has also at least 2 elements)
                if left_idx < pivot_position:
                    stack.push((left_idx, pivot_position))
        else:
            # Store indices of the longer part at first if has at least 2 elements
            if left_idx < pivot_position:
                stack.push((left_idx, pivot_position))
                # Then store indices of a shorter part (as it will produce less parts after sorting)
                # (if has at least 2 elements and we are sure the longer part has also at least 2 elements)
                if pivot_position + 1 < right_idx:
                    stack.push((pivot_position + 1, right_idx))
        
        
def _partition(arr, left_idx, right_idx):
    pivot = arr[left_idx]
    
    # Partition an array into 2 subarrays of elements lower than or
    # equal to a pivot and of elements greater than a pivot (in this
    # partition algorithm pivot isn't placed on a fixed position but
    # can be also swapped like all the remaining values)
    i = left_idx - 1
    j = right_idx + 1
    while True:
        i += 1
        while arr[i] < pivot: i += 1
            
        j -= 1
        while arr[j] > pivot: j -= 1
        
        if i < j:
            _swap(arr, i, j)
        else:
            return j  # Return a pivot position after the last swap

    
def _swap(arr, i, j):
    arr[i], arr[j] = arr[j], arr[i]

###### Kilka testów

In [6]:
test_sort(quick_sort, samples=100)

TEST #1:
Before sorting: [-29, -62, -5, -1, 86, 67, 81, 13, -2, -93, -87, 61, 51, -95]
After sorting: [-95, -93, -87, -62, -29, -5, -2, -1, 13, 51, 61, 67, 81, 86]
Expected result: [-95, -93, -87, -62, -29, -5, -2, -1, 13, 51, 61, 67, 81, 86]
Test PASSED
Current passed-to-tested ratio: 1/1

TEST #2:
Before sorting: [46, -49, -2, 82]
After sorting: [-49, -2, 46, 82]
Expected result: [-49, -2, 46, 82]
Test PASSED
Current passed-to-tested ratio: 2/2

TEST #3:
Before sorting: [-1, -38, 100, -16, -78, 75, -29, -30, -67, -98, -10, 92, -7, -89, 62, -11, -51, -84, -1, -55, 86, 22, 62, -81, -24, 40, -24]
After sorting: [-98, -89, -84, -81, -78, -67, -55, -51, -38, -30, -29, -24, -24, -16, -11, -10, -7, -1, -1, 22, 40, 62, 62, 75, 86, 92, 100]
Expected result: [-98, -89, -84, -81, -78, -67, -55, -51, -38, -30, -29, -24, -24, -16, -11, -10, -7, -1, -1, 22, 40, 62, 62, 75, 86, 92, 100]
Test PASSED
Current passed-to-tested ratio: 3/3

TEST #4:
Before sorting: [-60, 61, 100, 16, -89, -49, 20, 31, -9

### Implementacja algorytmu #3 (ze stosem w postaci własnej listy odsyłaczowej zaimplementowanym funkcyjnie)

#### Implementacja stosu

In [7]:
class Node:
    def __init__(self, val=None):
        self.val = val
        self.next = None
    
    
def stack_create() -> 'stack top (sentinel)':
    head = Node()  # A sentinel node
    return head
    
    
def stack_push(stack_top: 'stack top (sentinel)', val: object):
    node = Node(val)
    node.next = stack_top.next
    stack_top.next = node
    
    
def stack_pop(stack_top: 'stack top (sentinel)') -> object:
    if not stack_top.next:
        raise IndexError('pop from an empty stack')
    removed = stack_top.next.val
    stack_top.next = stack_top.next.next
    return removed


def stack_is_empty(stack_top: 'stack top (sentinel)') -> bool:
    return not stack_top.next

#### Implementacja algorytmu sortowania

In [8]:
def quick_sort(arr):
    if len(arr) < 2: return
    stack = stack_create()
    stack_push(stack, (0, len(arr)-1))
    
    while not stack_is_empty(stack):
        # Take the last indices pair out of the stack
        left_idx, right_idx = stack_pop(stack)
        
        pivot_position = _partition(arr, left_idx, right_idx)

        if pivot_position - left_idx < right_idx - pivot_position:
            # Store indices of the longer part at first if has at least 2 elements
            if pivot_position + 1 < right_idx:
                stack_push(stack, (pivot_position + 1, right_idx))
                # Then store indices of a shorter part (as it will produce less parts after sorting)
                # (if has at least 2 elements and we are sure the longer part has also at least 2 elements)
                if left_idx < pivot_position:
                    stack_push(stack, (left_idx, pivot_position))
        else:
            # Store indices of the longer part at first if has at least 2 elements
            if left_idx < pivot_position:
                stack_push(stack, (left_idx, pivot_position))
                # Then store indices of a shorter part (as it will produce less parts after sorting)
                # (if has at least 2 elements and we are sure the longer part has also at least 2 elements)
                if pivot_position + 1 < right_idx:
                    stack_push(stack, (pivot_position + 1, right_idx))
        
        
def _partition(arr, left_idx, right_idx):
    pivot = arr[left_idx]
    
    # Partition an array into 2 subarrays of elements lower than or
    # equal to a pivot and of elements greater than a pivot (in this
    # partition algorithm pivot isn't placed on a fixed position but
    # can be also swapped like all the remaining values)
    i = left_idx - 1
    j = right_idx + 1
    while True:
        i += 1
        while arr[i] < pivot: i += 1
            
        j -= 1
        while arr[j] > pivot: j -= 1
        
        if i < j:
            _swap(arr, i, j)
        else:
            return j  # Return a pivot position after the last swap

    
def _swap(arr, i, j):
    arr[i], arr[j] = arr[j], arr[i]

###### Kilka testów

In [9]:
test_sort(quick_sort, samples=100)

TEST #1:
Before sorting: [-44, 42, -5, -80, -56, 71, 71, -49, -34, -19, 51, -6, 71, 92]
After sorting: [-80, -56, -49, -44, -34, -19, -6, -5, 42, 51, 71, 71, 71, 92]
Expected result: [-80, -56, -49, -44, -34, -19, -6, -5, 42, 51, 71, 71, 71, 92]
Test PASSED
Current passed-to-tested ratio: 1/1

TEST #2:
Before sorting: [-52, 27, -66, -93, 54, -3, -94, 22]
After sorting: [-94, -93, -66, -52, -3, 22, 27, 54]
Expected result: [-94, -93, -66, -52, -3, 22, 27, 54]
Test PASSED
Current passed-to-tested ratio: 2/2

TEST #3:
Before sorting: [27, -14, 73, -55, -73, -2, 36, 51, 79, -18, 51, 100, -74, -82, -55, -18, 93, -65, 23, -24, -43, 50]
After sorting: [-82, -74, -73, -65, -55, -55, -43, -24, -18, -18, -14, -2, 23, 27, 36, 50, 51, 51, 73, 79, 93, 100]
Expected result: [-82, -74, -73, -65, -55, -55, -43, -24, -18, -18, -14, -2, 23, 27, 36, 50, 51, 51, 73, 79, 93, 100]
Test PASSED
Current passed-to-tested ratio: 3/3

TEST #4:
Before sorting: [26, 74, 67, -72, -6, -81, -63, -60, 75, 81, -90, -63,